<a href="https://colab.research.google.com/github/datascience-uniandes/data-quality-tutorial/blob/master/data-quality-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality tutorial

Jose Julian Sanabria Mejia <br> Cod: 202224837 <br> jj.sanabria@uniandes.edu.co

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
Dataset: Homicides Colombia  
Source: [datos.gov.co](datos.gov.co)
  
Last update: September, 2022

In [1]:
!pip install pylev

In [2]:
# Importing required libraries

import re
from random import randint
from datetime import datetime

import numpy as np
import pandas as pd

import pylev

In [3]:
# Parameter for showing all columns when printing a dataframe
pd.set_option('display.max_columns', None)

In [4]:
# Loading data
homicides_df = pd.read_csv("./data/homicides.csv")

In [5]:
# Printing the dataset dimensions
homicides_df.shape

(12400, 22)

In [6]:
# Printing column data types
homicides_df.dtypes

FECHA                 object
DEPARTAMENTO          object
MUNICIPIO             object
DIA                   object
HORA                  object
BARRIO                object
ZONA                  object
CLASE DE SITIO        object
ARMA O MEDIO          object
MOVIL VICTIMA         object
MOVIL AGRESOR         object
EDAD                 float64
GENERO                object
ESTADO CIVIL          object
CLASE EMPLEADO        object
PROFESION             object
ESCOLARIDAD           object
PAIS NACE             object
CODIGO DANE          float64
AÑO DE NACIMIENTO    float64
CÉDULA                object
CORREO                object
dtype: object

In [7]:
homicides_df.head()

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,01/01/2021 12:00:00 AM,ANTIOQUIA,AMAGÁ,Jueves,6:00,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,01/01/2021 12:00:00 AM,ANTIOQUIA,BARBOSA,Jueves,9:00,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,01/01/2021 12:00:00 AM,ANTIOQUIA,EL BAGRE,Jueves,19:00,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,01/01/2021 12:00:00 AM,ANTIOQUIA,JARDÍN,Jueves,11:20,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,01/01/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Juees,15:00,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co


In [10]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [11]:
# Applying the validation to all values in column
homicides_df["FECHA"].apply(dateparse)

# IT IS EXPECTED TO HAVE AN ERROR INTENTIONALLY

ValueError: time data 'TOTAL' does not match format '%m/%d/%Y %H:%M:%S %p'

In [12]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [13]:
# Using the function for validation
homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]

# THE LAST ROW IS CAUSING THE ERROR

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
12399,TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Deleting a row by its index
homicides_df.drop(12399, inplace = True)

In [15]:
# Trying to parse the datetime string again
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [16]:
# Counting homicides by hour
homicides_df["FECHA"].dt.hour.value_counts()

# All datetime hour parts are the same

12    12399
Name: FECHA, dtype: int64

In [17]:
# The homicide hour is available in a different column!
# Merging both columns
homicides_df["FECHA"] = homicides_df["FECHA"].astype(str).apply(lambda x: x[:11]) + homicides_df["HORA"]

In [18]:
del homicides_df["HORA"]

In [19]:
homicides_df["FECHA"].head()

0     2021-01-01 6:00
1     2021-01-01 9:00
2    2021-01-01 19:00
3    2021-01-01 11:20
4    2021-01-01 15:00
Name: FECHA, dtype: object

In [20]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [21]:
# Applying the expression
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [22]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [23]:
homicides_df.head()

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,2021-01-01 06:00:00,ANTIOQUIA,AMAGÁ,Jueves,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,2021-01-01 09:00:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20:00,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00:00,ANTIOQUIA,MEDELLÍN (CT),Juees,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co


In [24]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ',
       'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'META',
       'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO', 'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA',
       'GUAINÍA', 'VICHADA', 'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [25]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array(['SAN ANDRÉS', 'N. DE SANTANDER'], dtype=object)

<span style="color:red">TODO: Replace the values identified as error to a valid value from the dictionary</span>

<span style="color:red">Hint: You can use the replace() pandas function</span>

In [27]:
# Vuelvo numérica la variable categórica sex
homicides_df['DEPARTAMENTO'] = homicides_df['DEPARTAMENTO'].replace(['SAN ANDRÉS', 'N. DE SANTANDER'],['SAN ANDRÉS Y PROVIDENCIA', 'NORTE DE SANTANDER'])

In [28]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array([], dtype=object)

In [29]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep = False)]

In [30]:
duplicates_by_cedula.shape

(1627, 21)

In [31]:
# Showing some examples
duplicates_by_cedula.sort_values("CÉDULA", ascending = True).head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
4041,2021-05-03 23:30:00,CAUCA,EL TAMBO,Domingo,LA VICTORIA,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,19256000.0,1970.0,10-048,aitufn1227@gmail.com
3810,2021-04-26 03:00:00,ANTIOQUIA,SAN JERÓNIMO,Domingo,LA PLAYA,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",CUERDA/SOGA/CADENA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5656000.0,1990.0,10-048,ibbcpu2509@unidatos.edu.co
693,2021-01-19 05:30:00,ANTIOQUIA,SALGAR,Lunes,LA HABANA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5642000.0,57.0,10-079,unhoqj1172@unidatos.edu.co
10796,2021-11-20 03:00:00,VALLE,CALI (CT),Viernes,POTRERO GRANDE E21,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,21.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1994.0,10-079,iknnoj8430@unidatos.edu.co
2386,2021-03-14 02:00:00,META,FUENTE DE ORO,Sábado,VEREDA PUERTO NUEVO,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA BLANCA,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,50287000.0,1973.0,10-255,afompq7113@unidatos.edu.co
138,2021-01-02 06:20:00,CAUCA,SOTARA,Viernes,CENTRO,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,22.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19760000.0,1993.0,10-255,cobgqs8819@unidatos.edu.co


<span style="color:red">TODO: Delete records with "CÉDULA" duplicated</span>

<span style="color:red">Hint: You can use drop_duplicates() pandas function.</span>

In [ ]:
df2 = df.drop_duplicates(subset=["Courses", "Fee"], keep=False)

In [ ]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [ ]:
cedula_malformed.shape

In [ ]:
cedula_malformed.head(6)

<span style="color:red">TODO: Fix the malformed "CÉDULA" values.</span>

<span style="color:red">TODO: Make something similar to check and fix the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains are allowed)</span>

In [ ]:
# Showing different values for "DIA" column
homicides_df["DIA"].unique()

In [ ]:
# Calculating the distance between two words using the Levenshtein method
pylev.levenshtein('sábado', 'sabaod')

In [ ]:
pylev.levenshtein('sábado', 'viernes')

<span style="color:red">TODO: Create a function to fix the digitation errors for "DIA" column.</span>

In [ ]:
homicides_df["DIA"].unique()

In [ ]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].head(10)

<span style="color:red">TODO: Fix the "AÑO DE NACIMIENTO" column using the column "EDAD".</span>